<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Residuals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residuals

Authors: Madeline Keep, Izabella Kornak, Hannah Weinstein

[/Monroe/]: # (Good start. Somehow you missed that this is a Sedimentation plant and doesn't include filters. That will change your starting assumptions. Use the previous design challenge to help you get started with the volume that you need to process. The SOW also has estimates and they are different from our estimates and so one of your tasks is to reconcile our calculations with the SOW estimates. )

[/Monroe/]: # (I'd suggest that estimating volumes of sludge to be treated and finding equipment manufacturers would be top priorities.)


In [0]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.5MB/s 
     |████████████████████████████████| 163kB 7.8MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=e2217fe803f88209d3cfc3a95dd4853dc375ea196fd41c34edd42705067cb6f0
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=df6cae9f36be7c5e19338871e5e7cb1308f1c4abe27081891998b60726e88c59
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.2MB/s 


In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-20 13:19:47--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1604 (1.6K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   1.57K  --.-KB/s    in 0s      

2020-04-20 13:19:47 (262 MB/s) - ‘Fairmont.py’ saved [1604/1604]



# Task Definition

Our goal is to develop a complete analysis of the composition, management, and disposal of residuals from the proposed treatment process.
According to the Fairmont Treatment Plant’s Scope Of Work document, settled solids from the sedimentation process will be mechanically thickened in an on site facility. The preliminary evaluation suggests using equalization basins, rapid mixers, then settling tanks, and introducing coagulants to form and settle out thickened solids. The residual solids will then be further mechanically dewatered at a separate facility, then can be temporarily stored in a sludge hopper (with the capacity to store a minimum of a week’s worth of sludge) before they are transferred via truck to be disposed off-site. The leftover water will be recycled to the head of the Fairmont sedimentation plant. 

A preliminary evaluation has concluded that that centrifuges are most effective active dewatering technology. We will develop a design for the thickening facility and dewatering facility and evaluate their cost and effectiveness. Our thickening facility will take the collected sludge and thicken it, and also even out the flow before the sludge before it enters the dewatering facility. These are expected to be similar to the coagulation and flocculation tanks. The dewatering facility will be composed of centrifuges that will further dewater the sludge to create a sludge cake. There also must be a distribution mechanism for the sludge cake to be moved out of the facility.

We will also develop a more exact estimate of the expected residual output of the process, and determine a minimum capacity for our residual treatment train.

[/Monroe/]: # (Is the thickening facility separate from the dewatering facility?)
[/Monroe/]: # (I suspect that evaluating cost is beyond the scope of what we can do in the next few weeks. See if you can find centrifuges for this job and find out how much sludge they can process.)

[/Monroe/]: # (Why is evening out the flow necessary? What is the constraint that sets the size of the sludge flow equilization tank? What failure modes might you be designing for here?)


# Executive Summary 

Our main constraints will rely on the amount of sludge that needs to be treated. Those calculations will depend on the treatment specifications of the plant. This will determine the characteristics of the sludge, and allow us to analyze potential solutions with concrete estimations of the costs of management. Our design solution will develop as we calculate these values



# Constraints
*(Relevant regulations, client expectations, site and budget, process capabilities including process design equations, ease of use, operation requirements,maintainence requirements)*

* Amount of material that must be disposed of
* Transportation costs
* Disposal costs
* Discharge requirements/pollution standards
* Residuals management systems may limit overall plant capacity if not designed and operated properly
* Toxicity of residuals
* Specific resistance of sludge
* Amount of water used
  * (“The quantity of waste washwater for both granular and membrane filters will typically comprise from 2 to 5 percent of the total amount of water processed”-MWH)
* Capacity of washwater facility to to accept high, intermittent flows from backwashing 
* Unit filter rn volumes (UFRV)
* Unit backwash volume (UBMV)
* Effective filtration rate and recover or production efficiency of filter
  * (“Typically, waste washwater quantities are 8 m3/m2 (200 gal/ft2). To achieve a filter production efficiency of 95 percent, the UFRV would have to be at least 200 m3/m2 (5000 gal/ft2) a run. At a filtration rate of 12.2 m/h (5 gpm/ft2), a filter run would have to last at least 1000 min between backwash cycles.” -MWH)

[/Monroe/]: # (The Fairmont Sedimentation Plant does NOT have granular or membrane filters. Rewrite the previous section taking into account that this is a Sedimentation Plant!)

# Physics

* At what steps we should treat/recycle residuals
* Typically, waste washwater quantities are 8 m3/m2 (200 gal/ft2). To achieve a filter production efficiency of 95 percent, the UFRV would have to be at least 200 m3/m2 (5000 gal/ft2) a run. At a filtration rate of 12.2 m/h (5 gpm/ft2), a filter run would have to last at least 1000 min between backwash cycles. -MWH
* The TSS/turbidity ratio for most water sources will vary between 1 and 2, with a typical value being about 1.4. For turbidities less than 10 NTU the ratio is nearly equal to 1. -MWH
* Percentage of plant flow that each type of residual contributes to (Table 21-1 MWH)
* Table 21-3 (Physical, chemical, and biological properties used to characterize water treatment plant residuals)

[/Monroe/]: # (Review the Fairmont intro assignment. The goal of that assignment was to help you start thinking about this design. Use the calculations you did there to find mass and volumetric flow rates.)

# Table listing potential strategies to meet the dewatering constraints and treatment objectives 


| strategy | advantages  | disadvantages |
|---|---|---|
| Centrifuge | high speed dewatering | high energy input | 
| | 25% and 35% solids by weight.| standby unit needed in some cases |
||works for all sludge types|very loud to run|
||cost less to operate than filter press||
| Plate and Frame Filter Press | stackable, cake-like solids | high potential for blockage and damage | 
| | 40% and 70% solids by weight | potential for leakage |
| |can operate on a continual basis | |
| Belt Filter Press | effective on high-volume waste streams  | difficult maintenance |  |
| | 18% and 25% solids by weight. | suitable for high sludge moisture content|
| | low energy and space taken up| |
||10 year service life||

[/Monroe/]: # (How much energy is required to process sludge using a centrifuge? Are the centrifuges batch or continuous flow? Where did you obtain the information?)

#Table listing potential strategies to meet sludge handling constraints and objectives

| strategy | advantages  | disadvantages |
|---|---|---|
|Sludge Detention Ponds| can further thicken sludge | 280,000 sq ft of land required|
| | lowers volume of sludge in dumping site| high cost of $46000/day| 
|Sludge Processing Facility| converts hazardous waste into nonhazardous | high cost|
| | has potential to create new materials | still in research phase, not usable|
|Off-Site Dumping | lower cost | hazardous waste dumping | 
| | No extra land or equipment needed | highest volume of waste produced |

[/Monroe/]: # (We are using Colab notebooks so you can do the calculations in the notebook. There shouldn't be calculated values that don't have any documentation of where they came from. The easy way to do this is to add a code cell and do the calculation there. Done. Easy. Documented! Is there enough land on the site for sludge drying ponds?)

[/Monroe/]: # (How are you proposing to convert Arsenic in to a nonhazardous waste? I realize you don't mean to suggest that, but you list that as an option! Fix the table to not include alchemy!)

# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [0]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design
5,rapid mixer (mech),Dynamix,still,pending,https://www.dynamixinc.com,https://www.dynamixinc.com/industries/waste-wa...,4/16/2020,Lynn Li,NaN
6,rapid mixer (mech),Meurer Research,still,pending,http://www.meurerresearch.com,http://www.meurerresearch.com/products/mixer/,4/16/2020,Lynn Li,NaN
7,rapid mixer (mech),Philadelphia Mixing Solutions,still,pending,https://www.philamixers.com,https://www.philamixers.com/industries/water-w...,4/16/20,Lynn Li,NaN


# Shared assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

Challenges include trying to reduce the volume of residuals within the plant. This should be done by concentrating wastes as much as possible. This leftover water should be recycled back into the front end of the plant. Also, considering what to do with the residuals can be difficult due to limited space availability of dumping in Los Angeles. Another challenge includes finding safe ways to deal with Arsenic in the residuals, since it is a hazardous waste, and presents itself in high concentrations through the plant inflow. The management of liquid waste streams is also going to be challenging, as our team will try to recycle as much water as possible, and the constituents in the return flow are difficult to manage. 

Next Steps include contacting the other teams to decide which treatment trains we should focus on removing residuals from, and researching more on the types of contaminants in typical inflow waters in the area. This could help our team in determining what types of chemicals and processes will be used in getting rid of contaminants, and the amount of residual flows we will need to focus on. 
We also will need to research the regulatory standards out in place for the contaminants we will be working to remove, in order to understand how and where they can be disposed of. 
Another large part of our next steps includes researching methods of equations for mass of wastes, figure out who manufacturers are of units that process the solids, and suggest centrifuges. reasonable task might be to look at the top options for consolidating solids in addition to centrifuges (other good options) see if you agree if that is the top choice. Identify the "why" of the  technology approach. Helping to think about the overall task, not necessarily this week.


# California Waste Discharge Requirements

"Section 13260(a) of the California Water Code requires that any person discharging waste or proposing to discharge waste within any region, other than to a community sewer system, that could affect the quality of the waters of the State, must file a report of waste discharge. All waste discharge requirements issued by the Regional Water Board include self-monitoring programs requiring the waste discharger to collect pertinent water quality data and to submit it to the Regional Water Board for evaluation of compliance with waste discharge requirements."

The treatment of drinking water results in liquid and solid wastes that contain concentrated contaminants such as arsenic. Faimont must dispose of their residuals in a way that complies with California law and minimizes environmental impact. Raw water characteristics that may impact the disposal optins of residuals include:

• Excessively high or low pH;

• High total suspended solids (TSS);

• High total dissolved solids (TDS);

• High concentrations of heavy metals, including arsenic, lead, and aluminum;

• High concentrations of competing ions, including fluoride, sodium, sulfate,
chloride, and other salts concentrations;

• High concentrations of radionuclides and daughter products.

[source](https://www.epa.gov/sites/production/files/2015-09/documents/train6-residuals.pdf)


To dispose of residuals in a solid waste landfill, the residuals must be nonhazardous, otherwise they must be disposed of in a hazardous waste landfill. Liquid residuals cannot be disposed of in a land disposal unit. To discharge resdiuals directly to a body, the residuals must be non-hazardous and meet National Pollutant Discharge Elimination System (NPDES) discharge requirements.

Depending on the concentrations of contanimants in a waste stream, intermediate processing may be neccessary before disposal. Types of intermediate processing for liquid waste includes flow equilization (detaining and mixing water system wastes before releasing residuals to a POTW), brine recycling (however caution must be exercised, as reused brine streams can possess higher levels of suspended solids and contaminants), pH neutralization for wastes streams with a significantly high or low pH, settling or gravity thickening (removes contaminants from liquid waste stream, resulting in a cleaner liquid waste and a concentrated sludge), evaporation (leaves solids and contaminants behind), and chemical precipitation. 

Intermediate precoessing options for solid wastes includes mechanical dewatering processes (centrifugal filter presses, vacuumassisted dewatering beds, belt filter presses, plate-and-frame filter presses), filter presses, non-mechanical dewatering processes (lagoons, settling basins, gravity thickening, evaporation ponds, drying beds), and thermal processing (incineration, pyrolysis).


For the most part, the dillution of hazardous waste in not recommended, as any mixture or derivative of a hazardous waste in also characterized as a hazardous waste. 


The national TC (toxicity characteristic) for arsenic is 5 mg/L. If the concentration of arsenic in residuals exceeds this value, the waste in considered hazardous and is not acceptable for direct discharge. However, this value can vary region-to-region. The TBLL (technically based local limit) for arsenic is 1 mg/L in San Jose, CA and 2 mg/L in Orange County, CA. 


Nonhazardous residuals produced from sorption processes or chemical precipitation may be disposed of in the following ways:

>Recycle backwash water to facility headwokrs, discharge backwash to recieving body or POTW, and dispose of solid wastes in landfill.  

Nonhazardous residuals produced from membrane processes may be disposed of in the following ways:

>Solid wastes disposed in landfill, liquid wastes may need pretreatment before discharge into POTW. 

# Health-based acceptable loading limites established under 40 CFR 503:


Pollutant | Ceiling concentration (mg/kg) | Cumulative pollutant loading rate (kg/hectare) | Monthly avg concentration (mg/kg) | Annual Pollutant loading rate (kg/hectare/365 day period)
--- | --- | --- | --- | ---
Arsenic| 75 | 41 | 41 | 2.0 
Cadmium | 85 | 39 | 39 | 1.9 
Copper | 4300 | 1500 | 1500 | 75 
Lead | 840 | 300 | 300 | 15 
Mercury | 57 | 17 | 17 | 0.85
Molybdenum | 75 | -- | -- | -- 
Nickel | 420 | 420 | 420 | 21
Selenium | 100 | 100 | 100 | 5.0 
Zinc | 7500 | 2800 | 2800 | 140


# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [0]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

The average flow to be treated at Fairmont is 206.8 megagallon / day or
The maximum flow to be treated at Fairmont is 465.3 megagallon / day
The average amount of energy used per liter of water is expected to be 118.1 joule / liter or 12.05 meter of potential energy
